In [1]:
!pip install delta-spark==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 119.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 155.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040922 sha256=3d159b25be69e2b7f4dd381c851d0796ee8ca160d1dfa640f487160ffd4b5420
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nhwyool/wheels/e7/e3/c8/c358dac750f2b6a4b03328d10e05a5c69501664bd6504b6c3e
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.3
    Uninstalling py4j-0.10.9.3:
      Successfully uninstalled py4j-0.10.9.3


In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [3]:
print(pyspark.__version__)

3.2.4


In [4]:

####### read from mongodb ########
url = 'mongodb://mongodb:27017/auto-mpg.auto'
# spark = (SparkSession
#          .builder
#          .master('local[*]')
#          .config('spark.driver.extraClassPath','/usr/local/spark-3.2.1-bin-hadoop3.2/jars/*')
#          .config("spark.mongodb.read.connection.uri",url)
#          .config("spark.mongodb.write.connection.uri", url)
#          .getOrCreate()
#          )
spark = (SparkSession
         .builder
         .master('local[*]')
         .config('spark.jars','/usr/local/spark-3.2.1-bin-hadoop3.2/jars/*')
         .config("spark.mongodb.read.connection.uri",url)
         .config("spark.mongodb.write.connection.uri", url)
         .getOrCreate()
         )
auto_df = spark.read.format("mongodb").load()
auto_df.createOrReplaceTempView("auto")
auto_df= spark.sql("SELECT * FROM auto")
auto_df.show()

+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+
|                 _id|acceleration|             carname|cylinders|displacement|horsepower|index|modelyear| mpg|origin|weight|
+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+
|6548f171500f4d6b0...|        12.0|chevrolet chevell...|        8|       307.0|     130.0|    0|       70|18.0|     1|3504.0|
|6548f171500f4d6b0...|        11.5|   buick skylark 320|        8|       350.0|     165.0|    1|       70|15.0|     1|3693.0|
|6548f171500f4d6b0...|        11.0|  plymouth satellite|        8|       318.0|     150.0|    2|       70|18.0|     1|3436.0|
|6548f171500f4d6b0...|        12.0|       amc rebel sst|        8|       304.0|     150.0|    3|       70|16.0|     1|3433.0|
|6548f171500f4d6b0...|        10.5|         ford torino|        8|       302.0|     140.0|    4|       70|17.0|     1|

In [5]:
import pyspark
from delta import *
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DeltaApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).master("spark://spark:7077").getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "FFD34B2AC56E76E8BB9E7EFD7D283")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "47A13EFAB1CA1F253FD6F56DEF769")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("spark.jars", "postgresql-42.2.14.jar")

In [6]:
# read from minio
df_manufacturers = spark.read \
        .format("json") \
        .option("inferSchema", "true") \
        .json("s3a://landing/manufacturers//*.json")

df_manufacturers.show()


+-------+--------------------+-------------+
|country|dt_current_timestamp| manufacturer|
+-------+--------------------+-------------+
|     us|       1699268792725|          amc|
|germany|       1699268792725|         audi|
|germany|       1699268792725|          bmw|
|     us|       1699268792725|        buick|
|     us|       1699268792725|     cadillac|
|     us|       1699268792725|        capri|
|     us|       1699268792725|    chevroelt|
|     us|       1699268792725|    chevrolet|
|     us|       1699268792725|        chevy|
|     us|       1699268792725|     chrysler|
|  japan|       1699268792725|       datsun|
|     us|       1699268792725|        dodge|
|  italy|       1699268792725|         fiat|
|     us|       1699268792725|         ford|
|     us|       1699268792725|           hi|
|  japan|       1699268792725|        honda|
|  japan|       1699268792725|        maxda|
|  japan|       1699268792725|        mazda|
|germany|       1699268792725|     mercedes|
|germany| 

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

#Join two DataFrames with an expression
# Add a manufacturers column, to join with the manufacturers list.
first_word_udf = udf(lambda x: x.split()[0], StringType())
df = auto_df.withColumn("manufacturer", first_word_udf(auto_df.carname))

# The actual join.
df_join = df.join(
    df_manufacturers,
    (df.manufacturer == df_manufacturers.manufacturer)
    | (df.mpg == df_manufacturers.manufacturer),
)
#drop column
df_final = df_join.drop("dt_current_timestamp")
colnames = df_final.columns
df_final = df_final.toDF(*map(str, range(len(colnames))))\
    .drop(str(len(colnames)-1))\
    .toDF(*colnames[:-1])
df_final.show()

+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+------------+-------+
|                 _id|acceleration|             carname|cylinders|displacement|horsepower|index|modelyear| mpg|origin|weight|manufacturer|country|
+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+------------+-------+
|6548f171500f4d6b0...|        12.0|chevrolet chevell...|        8|       307.0|     130.0|    0|       70|18.0|     1|3504.0|   chevrolet|     us|
|6548f171500f4d6b0...|        11.5|   buick skylark 320|        8|       350.0|     165.0|    1|       70|15.0|     1|3693.0|       buick|     us|
|6548f171500f4d6b0...|        11.0|  plymouth satellite|        8|       318.0|     150.0|    2|       70|18.0|     1|3436.0|    plymouth|     us|
|6548f171500f4d6b0...|        12.0|       amc rebel sst|        8|       304.0|     150.0|    3|       70|16.0|     1|

In [8]:
# jdbcDF = spark.read.format("jdbc"). \
# options(
#         url='jdbc:postgresql://10.0.0.241:5432/postgres', # jdbc:postgresql://<host>:<port>/<database>
#         user='postgres',
#         password='DebPostgres1!',
#         driver='org.postgresql.Driver').\
# load()

In [9]:
df_final.show()

+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+------------+-------+
|                 _id|acceleration|             carname|cylinders|displacement|horsepower|index|modelyear| mpg|origin|weight|manufacturer|country|
+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+------------+-------+
|6548f171500f4d6b0...|        12.0|chevrolet chevell...|        8|       307.0|     130.0|    0|       70|18.0|     1|3504.0|   chevrolet|     us|
|6548f171500f4d6b0...|        11.5|   buick skylark 320|        8|       350.0|     165.0|    1|       70|15.0|     1|3693.0|       buick|     us|
|6548f171500f4d6b0...|        11.0|  plymouth satellite|        8|       318.0|     150.0|    2|       70|18.0|     1|3436.0|    plymouth|     us|
|6548f171500f4d6b0...|        12.0|       amc rebel sst|        8|       304.0|     150.0|    3|       70|16.0|     1|

In [11]:
df_final.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/postgres") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "public.auto") \
    .option("user", "postgres").option("password", "Sup3rS3c3t").mode('overwrite').save()